In [6]:
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from pydantic import BaseModel


checkpointer = InMemorySaver()


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"


def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:
    user_name = config["configurable"].get("user_name")
    system_msg = f"You are a helpful assistant. Address the user as {user_name}."
    return [{"role": "system", "content": system_msg}] + state["messages"]

kwargs = {
    "api_key": "ollama",
    "base_url": "http://localhost:11434/v1/"
}
model = init_chat_model("qwen3:8b", model_provider="openai", **kwargs)

class WeatherResponse(BaseModel):
    conditions: str

In [7]:
agent = create_react_agent(model=model, 
                           tools=[get_weather], 
                           prompt=prompt, 
                           checkpointer=checkpointer,
                           response_format=WeatherResponse)

response = agent.invoke({"messages": [{
    "role": "user",
    "content": "what is the weather in Beijing"
}]},
                        config={"configurable": {
                            "user_name": "John Smith",
                            "thread_id": "1"
                        }})

for message in response["messages"]:
    print(message.content)

what is the weather in Beijing
<think>
Okay, the user is asking for the weather in Beijing. Let me check the tools available. There's a function called get_weather that takes a city parameter. Since the user mentioned Beijing, I need to call that function with the city set to Beijing. I'll make sure to format the tool call correctly within the XML tags.
</think>


It's always sunny in Beijing!
<think>
</think>

The weather in Beijing is sunny! 🌞 Let me know if you need more details.


In [8]:
response['structured_response']

WeatherResponse(conditions='sunny')